# psk PL imaging tool, with beta version PL-v for perovskite solar cells

In [1]:

# Import specific packages for using quantalux camera
import os
root_dir = 'C:\\Users\\FRG-users\\Documents\\GitHub\\Instruments\\FRG Hardware\\frghardware\\components\\frghardware\\components'
os.environ['PATH'] = os.path.join(root_dir, 'dlls', 'Native_64_lib') + os.pathsep + os.environ['PATH']

from thorlabs_tsi_sdk.tl_camera import TLCameraSDK, OPERATION_MODE
from thorlabs_tsi_sdk.tl_camera_enums import DATA_RATE


# import pickle
import numpy as np
import time
import cv2
import time
import h5py
import pandas as pd


%matplotlib notebook
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable

from tqdm import tqdm
import datetime

# Psk stuff
#                                           1. Qtx camera
from frghardware.keithleyjv import control3 #2. Keithley Load
from frghardware.components.CNI import CNI #3. 635 nm CNI laser

# Si stuff
# from frghardware.components.FLIR import FLIR #4. FLIR InGaAs Camera
# from frghardware.components.kepco import Kepco #5. Kepco Load
from frghardware.components.laser import Laser808 #6. 808nm Jenoptik Laser

# Commonly used stuff
from frghardware.components.tec import omega #7. Omega PID Controller
from frghardware.components.opfo import opfo #8. Optics Focus XY Stage
from frghardware.components.daqPL3 import PLDAQ3 #9. Thorlabs Photodiode 


today = datetime.datetime.now() # create a folder in Data that's today's date
path =  f"x{today.strftime('%Y%m%d')}" 
root = f"F:\\{path}"
if not os.path.exists(root):
    os.mkdir(root)

datafolder = os.path.join(root, 'Data')
if not os.path.exists(datafolder):
    os.mkdir(datafolder)


# Put quantalux here because I somehow can't import it from a .py file. ZJD 2024/04/02
class qtx:
    def __init__(self):
        
        pass
    
    def capture(self, exposure_t, NUM_FRAMES, MODE = 'FPS_30'):
        with TLCameraSDK() as sdk:
            available_cameras = sdk.discover_available_cameras()
            if len(available_cameras) < 1:
                print("no cameras detected")

            with sdk.open_camera(available_cameras[0]) as camera:
                image_height = camera.image_height_pixels
                image_width = camera.image_width_pixels

                raw = np.zeros((image_height, image_width, NUM_FRAMES))

                if MODE == "FPS_30": #Low read noise mode
                    camera.data_rate = DATA_RATE.FPS_30
                if MODE == "FPS_50": #High frame rate mode
                    camera.data_rate = DATA_RATE.FPS_50
                camera.exposure_time_us = exposure_t  # set exposure to 11 ms
                camera.frames_per_trigger_zero_for_unlimited = 0  # start camera in continuous mode
                camera.image_poll_timeout_ms = 80000  # 5 second polling timeout, don't expect to ever need it

                camera.arm(2) #what is this?
                camera.issue_software_trigger() # begin aquisition

                try:
                    for i in range(NUM_FRAMES):
                        frame = camera.get_pending_frame_or_null()
                        if frame is not None:
    #                         print("frame #{} received!".format(frame.frame_count))

                            frame.image_buffer  # .../ perform operations using the data from image_buffer

                            #  NOTE: frame.image_buffer is a temporary memory buffer that may be overwritten during the next call
                            #        to get_pending_frame_or_null. The following line makes a deep copy of the image data:
                            image_buffer_copy = np.copy(frame.image_buffer)
                            raw[:, :, i] = np.squeeze(image_buffer_copy)
                            if np.any(raw[:, :, i] >= 65535):
                                raise ValueError("The img contains a number greater than or equal to 65536, quiting...")
                                break
                        else:
                            print("Unable to acquire image, program exiting...")
                            exit()
                except False:
                    print("Something's wrong, troubleshoot quantalux.py")

                camera.disarm()
                avg = np.mean(raw, axis = 2)
                std = np.std(raw, axis = 2)

        #  Because we are using the 'with' statement context-manager, disposal has been taken care of.

        return avg, std, raw


    def stream(self):

        with TLCameraSDK() as sdk:
            available_cameras = sdk.discover_available_cameras()
            if len(available_cameras) < 1:
                print("no cameras detected")

            with sdk.open_camera(available_cameras[0]) as camera:
                camera.exposure_time_us = 11000  # set exposure to 11 ms
                camera.frames_per_trigger_zero_for_unlimited = 0  # start camera in continuous mode
                camera.image_poll_timeout_ms = 1000  # 1 second polling timeout

                camera.arm(2) #what is this?
                camera.issue_software_trigger() # begin aquisition

                try:
                    while True:
                        frame = camera.get_pending_frame_or_null()
                        if frame is not None:
                            print("frame #{} received!".format(frame.frame_count))
                            frame.image_buffer
                            image_buffer_copy = np.copy(frame.image_buffer)
                            numpy_shaped_image = image_buffer_copy.reshape(camera.image_height_pixels, camera.image_width_pixels)
                            nd_image_array = np.full((camera.image_height_pixels, camera.image_width_pixels, 3), 0, dtype=np.uint8)
                            nd_image_array[:,:,0] = numpy_shaped_image
                            nd_image_array[:,:,1] = numpy_shaped_image
                            nd_image_array[:,:,2] = numpy_shaped_image

                            cv2.imshow("Press [enter] to close window", nd_image_array)

                            if cv2.waitKey(1) == 13:
                                print("loop terminated")
                                break

                        else:
                            print("Unable to acquire image, program exiting...")
                            exit()
                except False:
                    print("Something's wrong, troubleshoot quantalux.py")

                cv2.destroyAllWindows()
                camera.disarm()

        #  Because we are using the 'with' statement context-manager, disposal has been taken care of.

        print("streaming completed")    

    
class control:
    # keithley allow 0-200 V, laser is the 635 nm CNI laser
    def __init__ (self):
        # hardware properties
        
        self.__laserON = False
        self.__keithleyON = False
        self.__cameraON = False
        
        # measurement settings
        self.bias = 0 #bias applied to sample
        self.laserpower = 0 #current supplied to laser ###may replace this with n_suns, if calibration is enabled
        self.vocmode = True # ZJD added 2024/05/26
        self.saturationtime = 0 # #current supplied to laser ###may replace this with n_suns, if calibration is enabled, not sure if i would have something like this
        self.numIV = 20 
        self.numframes = 50	#number of image frames to average
        self.__temperature = 25	#TEC stage temperature setpoint (C) during measurement
        self.temperatureTolerance = 0.2	#how close to the setpoint we need to be able to take a measurement (C)
        self.maxSoakTime = 60	# max soak time, in seconds, to wait for temperature to reach set point. If we reach this point, just go ahead with the measurement
        self.note = ''
#         self._spotMap = None	# optical power map of laser spot, used for PL normalization
        self._sampleOneSun = None # fractional laser power with which to approximate one-sun injection levels
        self._sampleOneSunJsc = None # target Jsc, matching of which is used for one-sun injection level is approximated
        self._sampleOneSunSweep = None # fractional laser power vs photocurrent (Isc), fit to provide one-sun estimate
        self.__previewFigure = None	#handle for matplotlib figure, used for previewing most recent image results
        self.__previewAxes = [None, None]	# handle for matplotib axes, used to hold the image and colorbar
        self.__backgroundImage = None

        self.outputDirectory = datafolder # changed by ZJD as compared to Rishi's code 2023/11/07
        self.sampleName = None
        self.__dataBuffer = [] # buffer to hold data files during sequential measurements of single sample. Held until a batch export
        
        # stage/positioning constatns for Psk solar cell side
        self.__sampleposition = (85, 165) # add in later #position where TEC stage is contered in camera FOV, um
        self.__detectorposition = (85, 105) #delta position between detector and sampleposition, um. The Thorlabs pd's position
        self.__fov = (120, 120) #dimensions of FOV, um

        # stage/positioning constatns for Si solar cell side
        
        # For PSK PLIV 
        self._scanSpeed_timestop = 1

        # PSK Camera parameters
        self.exposure_t = 30000 #in microseconds
        self.NUM_FRAMES = 50
        self.camera_MODE = 'FPS_30'

        # Params for saved load or laser
        self.load = None
        self.laser = None
        self.camera = None

        # stage constants for controlling opfo
        self._screw_pitch = 5 #mm
        self._stepper_angle = 1.8 #degree
        self._subdivision = 2
        self._pulse_equivalent = self._screw_pitch * self._stepper_angle / (360 * self._subdivision)
        self._actual_displacement = 23 #mm, user input here to set movement distance
        self._pulse_number = self._actual_displacement / self._pulse_equivalent #input this to the axis

        speed_value = 50 #user input here to change movement speed
        self._actual_speed = (speed_value + 1) * 22000 * self._pulse_equivalent / 720

        self.connect()
        
    @property
    def temperature(self):
            return self.__temperature

    @temperature.setter
    def temperature(self, t):
        self.tec.setpoint = t
        self.__temperature = t

    def connect(self):
        # Psk stuff
        self.qtx = qtx() # connect to qtx camera
        self.keithley = control3.Control() # connect to Keithley 2401
        self.cni = CNI() # Connect to CNI laser
        
        # Si stuff
#         self.flir = FLIR()# connect to FLIR camera
#         self.kepco = Kepco()# connect to Kepco
# #         self.kepco.set(voltage=0)   # set voltage to 0, seems to solve current compliance issues
        self.jenoptik = Laser808()
  
        # Commonly used stuff
        self.daq3 = PLDAQ3() # connect to NI-USB6000 DAQ
        self.opfo = opfo() # connect to FRG stage
        self.tec = omega() # connect to omega PID controller, which is driving the TEC stage.
        
    
    def disconnect(self): #not tested, ZJD 2024/07/07

        """
        no qtx disconnect (ZJD hasn't checked yet)
        """
        
        print('qtx is disconencted as long as it is not taking images or streaming')
        
        try:
            self.keithley.disconnect()
        except:
            print('Could not disconnect Keithley load')

        try:
            self.cni.disconnect()
        except:
            print('Could not disconnect CNI laser')
            
            
#         try:
#             self.flir.disconnect()
#         except:
#             print('Could not disconnect FLIR camera')
#         try:
#             self.kepco.disconnect()
#         except:
#             print('Could not disconnect Kepco')            
#         try:
#             self.jenoptik.disconnect()
#         except:
#             print('Could not disconnect jenoptik/OSTech Laser')           
            
        try: 
            self.daq3.disconnect()
        except:
            print('Could not disconnect DAQ')
        try:
            self.opfo.disconnect()
        except:
            print('Could not disconnect opfo stage')
        try:
            self.tec.disconnect()
        except: 
            print('Could not disconnect TEC controller')
        

    ### basic use functions

    def setMeas(self, bias = None, laserpower = None, suns = None, load = None, laser = None, saturationtime = None, temperature = None, numIV = None, numframes = None, note = ''):
        """
        Uses 635nm laser and keithley for PSK imaging for now. ZJD 2024/07/07
        """

        if bias is None:
            bias = self.bias

        if laserpower is None:
            if suns is None:
                laserpower = self.laserpower

            else:
                if self._sampleOneSun is None:
                    print('Error: can\'t use "suns =" -  please run .findOneSun to calibrate one-sun power level for this sample.')
                    return False
                else:
                    laserpower = suns * self._sampleOneSun
                    if (laserpower > 1) or (laserpower < 0):
                        maxsuns = 1/self._sampleOneSun
                        print('Error: {0} suns is out of range! Based on laser power and current sample, allowed suns range = 0 - {1}.'.format(suns, maxsuns))
                        if laserpower > 1:
                            print('Setting to max laser power ({0} suns)'.format(maxsuns))
                            laserpower = 1

                        else:
                            print('Setting laser off')
                            laserpower = 0
        
        if saturationtime is None:
            saturationtime = self.saturationtime
        if temperature is None:
            temperature = self.__temperature
        if numIV is None:
            numIV = self.numIV
        if numframes is None:
            numframes = self.numframes 
            
#         if load is None:
#             print('No load, select keithley or kepco')
#             return
        
#         if load == 'keithley':
        self.load = load#????????
        result = self.keithley.set_voltage(voltage = bias) #onoff added by ZJD 2023/12/07

#         if load == 'kepco':
#             self.load = load
#             ## Add the insertion of command later

        
        if result:
            self.bias = bias
            self.keithley.on()# added by ZJD to troubleshoot jsc > vpplied
            self.__keithleyON = True
        else:
            print('Error setting keithley')
            # return False

        print(laserpower)
        """
        choose the laser to use
        """
#         if laser is None:
#             print('Select laser, cni or jenoptik')
#             return

#         if laser == 'cni':
        result = self.cni.set(laser_curr = laserpower)

#         if laser == 'jenoptik':
#             result = self.jenoptik.set(power = laserpower)

        if result:
            self.laserpower = laserpower
            self.laser = laser#????????
        else:
            print('Error setting laser')
            # return False


        result = self.tec.setSetPoint(temperature)
        if result:
            self.__temperature = temperature
        else:
            print('Error setting TEC temperature')
            return False


        self.numIV = numIV
        self.numframes = numframes
        self.note = note

    def takeMeas(self, camera = None, laser = None, load = None, lastmeasurement = True, preview = True, VocMode = False, imputeHotPixels = False):
        """
        takes a measurement with settings stored in method (can be set with .setMeas()).
        measurement settings + results are appended to .__dataBuffer

        if .__dataBuffer is empty (ie, no measurements have been taken yet), takeMeas() will 
        automatically take a 0 bias, 0 laser power baseline measurement before the scheduled
        measurement.
        """

#         if camera is None:
#             print('No camera, input camera as qtx or flir')
#             return
#         self.camera = camera

#         if laser is None:
#             laser = self.laser
#         if load is None:
#             load = self.load


        if len(self.__dataBuffer) == 0: # sample is being measured for the first time, take a baseline image
            print('New sample: taking a 0 bias, 0 illumination baseline image.')
            # store scheduled measurement parameters
            savedlaserpower = self.laserpower
            savedbias = self.bias
            savednote = self.note
            savedvocmode = self.vocmode
            savedload = self.load
            savedlaser = self.laser

            # take a 0 bias, 0 laserpower measurement, append to .__dataBuffer
            self.setMeas(bias = 0, laserpower = 0, note = 'automatic baseline image')
            print('keithley status', self.__keithleyON)
            measdatetime = datetime.datetime.now()
            temperature = self.tec.getTemperature()

            """
            Choose the camera to use
            """
#             if camera == 'qtx':
            im, _, _ = self.qtx.capture(exposure_t = self.exposure_t, NUM_FRAMES = self.NUM_FRAMES, MODE = self.camera_MODE) 
#             if camera == 'flir':
#                 im, _, _ = self.flir.capture(frames = self.numframes, imputeHotPixels = imputeHotPixels)

#             if load == 'keithley':
            v, i = self.keithley.read()# counts = self.numIV # i output is NOT multiplied by -1 here
#             self.__keithleyON = True # in keithley, .read turns on keithely, 02/01/2024
            
            irradiance = self._getOpticalPower()#what?
            temperature = (temperature + self.tec.getTemperature()) / 2	#average the temperature from just before and after the measurement. Typically averaging >1 second of time here.
            temperature_raw = self.tec.getTemperature()
            temperature_setpoint = self.tec.getSetPoint()
            meas = {
                'sample': 	self.sampleName,
                'note':		self.note,
                'date': 	measdatetime.strftime('%Y-%m-%d'),
                'time':		measdatetime.strftime('%H:%M:%S'),
                'cameraFOV':self.__fov,
                'bias':		self.bias,
                'laserpower': self.laserpower,
                'saturationtime': self.saturationtime,
                'numIV':	self.numIV,
                'numframes':self.numframes,
                'v_meas':	v,
                'i_meas':	i,
                'image':	im,
                'image_bgcorrected': im-im,
                'irradiance_ref': irradiance, 
                'temperature':	temperature,
                'temperature_raw': temperature_raw,
                'temperature_setpoint': temperature_setpoint,
                'exposure_time':self.exposure_t,
            }
            self.__dataBuffer.append(meas)
            self.__backgroundImage = im #store background image for displaying preview

            # restore scheduled measurement parameters + continue 	
            self.setMeas(bias = savedbias, laserpower = savedlaserpower, note = savednote)#???????? where s voc mode
            self.__keithleyON = False # When set_voltage is called in setMeas, keithley is turned OFF


        if not self.__laserON and self.laserpower > 0 :#and laser == 'cni'
            self.cni.on()
            self.__laserON = True
            print('laser on')

#         if not self.__laserON and self.laserpower > 0 and laser == 'jenoptik':
#             self.jenoptik.on()
#             self.__laserON = True
#             print('laser on')

        if not self.__keithleyON and not VocMode: # and load == 'keithley' 
            self.keithley.on()	#turn on the keithley source in short circuit (SC) mode, i.e. normal EL mode
            self.__keithleyON = True

        if VocMode:# and load == 'keithley'
            self.keithley._source_current_measure_voltage() # this is the open circuit part
            self.keithley.souce_current = 0
            self.keithley.keithley.enable_source() # Voc turned on 
            self.keithley.open_shutter()
            _ = self.keithley._measure()[0] # _meas() turns on Keithley
            
#             self.exposure_t = self.exposure_t_voc

            
        time.sleep(self.saturationtime)

        #take image, take IV meas during image
#         self._waitForTemperature()
        measdatetime = datetime.datetime.now()
        temperature = self.tec.getTemperature()
        
        
#         if camera == 'qtx':
        im, _, _ = self.qtx.capture(exposure_t = self.exposure_t, NUM_FRAMES = self.NUM_FRAMES, MODE = self.camera_MODE) 
#         if camera == 'flir':
#             im, _, _ = self.flir.capture(frames = self.numframes, imputeHotPixels = imputeHotPixels)

#         if load == 'keithley':
        v, i = self.keithley.read()#counts = self.numIV # k.read() also turns on keithley
    
        #pdb.set_trace()
        irradiance = self._getOpticalPower()
        temperature = (temperature + self.tec.getTemperature()) / 2	#average the temperature from just before and after the measurement. Typically averaging >1 second of time here.
        temperature_raw = self.tec.getTemperature()
        temperature_setpoint = self.tec.getSetPoint()
            
        if self.__laserON and lastmeasurement:# and laser == 'cni'
            self.cni.off()
            self.__laserON = False
#         if self.__laserON and lastmeasurement and laser == 'jenoptik':
#             self.jenoptik.off()
#             self.__laserON = False


        if self.__keithleyON and lastmeasurement:#changing_keithley_setting_next# and load == 'keithley'
            self.keithley.off()
            self.__keithleyON = False

        meas = {
            'sample': 	self.sampleName,
            'note':		self.note,
            'date': 	measdatetime.strftime('%Y-%m-%d'),
            'time':		measdatetime.strftime('%H:%M:%S'),
            'cameraFOV':self.__fov,
            'bias':		self.bias,
            'laserpower': self.laserpower,
            'saturationtime': self.saturationtime,
            'numIV':	self.numIV,
            'numframes':self.numframes,
            'v_meas':	v,
            'i_meas':	i,
            'image':	im,
            'image_bgcorrected': self._backgroundCorrection(im),
            'irradiance_ref': irradiance,
            'temperature_raw': temperature_raw,
            'temperature_setpoint': temperature_setpoint,
            'exposure_time':self.exposure_t,
        }
        self.__dataBuffer.append(meas)

        if preview:
            self.displayPreview(self._backgroundCorrection(im), v, i)
#         time.sleep(self._scanSpeed_timestop)
        
        return im, v, i#, self.__backgroundImage
####################################################### Fixed upon till this step, ZJD2024/07/07 #######################################################
    def displayPreview(self, img, v, i):
        def handle_close(evt, self):
            self.__previewFigure = None
            self.__previewAxes = [None, None]

        if self.__previewFigure is None:	#preview window is not created yet, lets make it
            plt.ioff()
            self.__previewFigure, self.__previewAxes[0] = plt.subplots()
            divider = make_axes_locatable(self.__previewAxes[0])
            self.__previewAxes[1] = divider.append_axes('right', size='5%', pad=0.05)
            self.__previewFigure.canvas.mpl_connect('close_event', lambda x: handle_close(x, self))	# if preview figure is closed, lets clear the figure/axes handles so the next preview properly recreates the handles
            plt.ion()
            plt.show()

        for ax in self.__previewAxes:	#clear the axes
            ax.clear()
        img_handle = self.__previewAxes[0].imshow(img)
        self.__previewFigure.colorbar(img_handle, cax = self.__previewAxes[1])
        self.__previewAxes[0].set_title('{0} V, {1} A, {2} Laser'.format(v, i, self.laserpower))
        self.__previewFigure.canvas.draw()
        self.__previewFigure.canvas.flush_events()
        time.sleep(1e-4)		#pause allows plot to update during series of measurements 
    
    
    def save(self, samplename = None, note = '', outputdirectory = None, reset = True):
        if len(self.__dataBuffer) == 0:
            print('Data buffer is empty - no data to save!')
            return False

        ## figure out the sample directory, name, total filepath
        if samplename is not None:
            self.sampleName = samplename

        if outputdirectory is not None:
            self.outputDirectory = outputdirectory
        if not os.path.exists(self.outputDirectory):
            os.mkdir(self.outputDirectory)

        fids = os.listdir(self.outputDirectory)
        sampleNumber = 1
        for fid in fids:
            if 'frgPL' in fid:
                sampleNumber = sampleNumber + 1

        todaysDate = datetime.datetime.now().strftime('%Y%m%d')

        if self.sampleName is not None:
            fname = 'frgPL_{0}_{1:04d}_{2}.h5'.format(todaysDate, sampleNumber, self.sampleName)
        else:
            fname = 'frgPL_{0}_{1:04d}_.h5'.format(todaysDate, sampleNumber)
            self.sampleName = ''

        fpath = os.path.join(self.outputDirectory, fname)

        numData = len(self.__dataBuffer)

        data = {}
        for field in self.__dataBuffer[0].keys():
            data[field] = []

        for meas in self.__dataBuffer:
            for field, measdata in meas.items():
                data[field].append(measdata)



        ## write h5 file

        with h5py.File(fpath, 'w') as f:
            # sample info
            info = f.create_group('/info')
            info.attrs['description'] = 'Metadata describing sample, datetime, etc.'

            temp = info.create_dataset('name', data = self.sampleName.encode('utf-8'))
            temp.attrs['description'] = 'Sample name.'

            temp = info.create_dataset('notes', data = np.array(note.encode('utf-8')))
            temp.attrs['description'] = 'Any notes describing each measurement.'

            date = info.create_dataset('date', data = np.array([x.encode('utf-8') for x in data['date']]))
            temp.attrs['description'] = 'Measurement date.'

            temp = info.create_dataset('time', data =  np.array([x.encode('utf-8') for x in data['time']]))
            temp.attrs['description'] = 'Measurement time of day.'


            # measurement settings
            settings = f.create_group('/settings')
            settings.attrs['description'] = 'Settings used for measurements.'

            temp = settings.create_dataset('vbias', data = np.array(data['bias']))
            temp.attrs['description'] = 'Nominal voltage bias set by Kepco during measurement.'

            temp = settings.create_dataset('notes', data = np.array([x.encode('utf-8') for x in data['note']]))
            temp.attrs['description'] = 'Any notes describing each measurement.'

            temp = settings.create_dataset('laserpower', data = np.array(data['laserpower']))
            temp.attrs['description'] = 'Fractional laser power during measurement. Calculated as normalized laser current (max current = 55 A). Laser is operated at steady state.'

            temp = settings.create_dataset('sattime', data = np.array(data['saturationtime']))
            temp.attrs['description'] = 'Saturation time for laser/bias conditioning prior to sample measurement. Delay between applying condition and measuring, in seconds.'

            temp = settings.create_dataset('numIV', data = np.array(data['numIV']))
            temp.attrs['description'] = 'Number of current/voltage measurements averaged by Kepco when reading IV.'

            temp = settings.create_dataset('numframes', data = np.array(data['numframes']))
            temp.attrs['description'] = 'Number of camera frames averaged when taking image.'

            temp = settings.create_dataset('tempsp', data = np.array(data['temperature_setpoint']))
            temp.attrs['description'] = 'TEC stage temperature setpoint for each measurement.'
            
            temp = settings.create_dataset('tempraw', data = np.array(data['temperature']))
            temp.attrs['description'] = 'TEC stage temperature setpoint for each measurement recorded by thermocouple.'


            if self.opfo.position()[0] is None:
                stagepos = self.__sampleposition
            else:
                stagepos = self.opfo.position()

            temp = settings.create_dataset('position', data = np.array(stagepos))
            temp.attrs['description'] = 'Stage position during measurement.'
            
            scanspeed_timestop = self._scanSpeed_timestop
            
            temp = settings.create_dataset('scanspeed_timestop', data = np.array(self._scanSpeed_timestop))
            temp.attrs['description'] = 'Time interval between each measurement. Added for PSK for the effect of scan speed' 
            
            exposure_time = self.exposure_t
            
            temp = settings.create_dataset('exposure_time', data = np.array(self.exposure_t))
            temp.attrs['description'] = 'Exposure time used for taking PLIV. Added for PSK pxiel measurement, as compared to module measurement.'           


            if self._sampleOneSun is not None:
                ### *1.42857−0.428571 comes from converting a [0.3pw = 0suns, 1pw = 1suns] laser to a [0.0pw = 0suns, 1pw = 1suns] measuring protocol
                ### example see below:
                # Let's say
                # c._sampleOneSun = 0.59

                # This is what feeds into c.setMeas
                # c._laserIntensityCorrection((np.linspace(0.2, 1, 5) * 0.7 + 0.3) * c._sampleOneSun)/c._sampleOneSun
                # returns: array([0.44      , 0.58      , 0.72      , 0.86779661, 1.        ])

                # This is what the c.setMeas feed into the laser
                # c._laserIntensityCorrection((np.linspace(0.2, 1, 5) * 0.7 + 0.3) * c._sampleOneSun)/c._sampleOneSun*c._sampleOneSun
                # returns: array([0.2596, 0.3422, 0.4248, 0.512 , 0.59  ])

                # This is what the suns are saved
                # (c._laserIntensityCorrection((np.linspace(0.2, 1, 5) * 0.7 + 0.3) * c._sampleOneSun)/c._sampleOneSun*c._sampleOneSun/c._sampleOneSun
                # ) *1.42857-0.428517
                # returns: array([0.2000538, 0.4000536, 0.6000534, 0.8111912, 1.000053 ])
                
                suns = [((x/self._sampleOneSun)*1.42857-0.428571) for x in data['laserpower']]
                temp = settings.create_dataset('suns', data = np.array(suns))
                temp.attrs['description'] = 'PL injection level in terms of suns. Only present if sample was calibrated with .findOneSun to match measured Isc to provided expected value, presumably from solar simulator JV curve.'

            # calibrations
            calibrations = f.create_group('/calibrations')
            calibrations.attrs['description'] = 'Instrument calibrations to be used for data analysis.'

            temp = settings.create_dataset('samplepos', data = np.array(self.__sampleposition))
            temp.attrs['description'] = 'Stage position (um)[x,y] where sample is centered in camera field of view'

            temp = settings.create_dataset('detectorpos', data = np.array(self.__detectorposition))
            temp.attrs['description'] = 'Stage position (um) [x,y] where photodetector is centered in camera field of view'

            temp = settings.create_dataset('camerafov', data = np.array(self.__fov))
            temp.attrs['description'] = 'Camera field of view (um) [x,y]'

            if self._sampleOneSunSweep is not None:
                temp = calibrations.create_dataset('onesunsweep', data = np.array(self._sampleOneSunSweep))
                temp.attrs['description'] = 'Laser current vs photocurrent, measured for this sample. Column 1: fractional laser current. Column 2: total photocurrent (Isc), NOT current density (Jsc). Only present if sample was calibrated with .findOneSun to match measured Isc to provided expected value, presumably from solar simulator JV curve.'

                temp = calibrations.create_dataset('onesun', data = np.array(self._sampleOneSun))
                temp.attrs['description'] = 'Fractional laser current used to approximate a one-sun injection level. Only present if sample was calibrated with .findOneSun to match measured Isc to provided expected value, presumably from solar simulator JV curve.'

                temp = calibrations.create_dataset('onesunjsc', data = np.array(self._sampleOneSunJsc))
                temp.attrs['description'] = 'Target Jsc (NOT Isc) used to approximate a one-sun injection level. Only present if sample was calibrated with .findOneSun to match measured Isc to provided expected value, presumably from solar simulator JV curve.'

            # raw data
            rawdata = f.create_group('/data')
            rawdata.attrs['description'] = 'Raw measurements taken during imaging'

            temp = rawdata.create_dataset('image', data = np.array(data['image']), chunks = True, compression = 'gzip')
            temp.attrs['description'] = 'Raw images acquired for each measurement.'

            temp = rawdata.create_dataset('image_bgc', data = np.array(data['image_bgcorrected']), chunks = True, compression = 'gzip')
            temp.attrs['description'] = 'Background-subtracted images acquired for each measurement.'

            temp = rawdata.create_dataset('v', data = np.array(data['v_meas']))
            temp.attrs['description'] = 'Voltage measured during measurement'

            temp = rawdata.create_dataset('i', data = np.array(data['i_meas']))
            temp.attrs['description'] = 'Current (not current density!) measured during measurement'

            temp = rawdata.create_dataset('irr_ref', data = np.array(data['irradiance_ref']))
            temp.attrs['description'] = 'Measured irradiance @ photodetector during measurement. Note that the photodetector is offset from the sample FOV. Assuming that the laser spot is centered on the sample, this value is lower than the true sample irradiance. This value should be used in conjunction with a .spotMap() calibration map.'			

            temp = rawdata.create_dataset('temp', data = np.array(data['temperature']))
            temp.attrs['description'] = 'Measured TEC stage temperature during measurement. This value is the average of two temperature measurements, just before and after the image/kepco readings/photodetector readings are made. These two values typically span >1 second'

            
            temp = rawdata.create_dataset('expt', data = np.array(data['exposure_time']))
            temp.attrs['description'] = 'Exposure time of image'

        print('Data saved to {0}'.format(fpath))
        if reset:
            self._sampleOneSun = None
            self._sampleOneSunSweep = None
            self._sampleOneSunJsc = None
            self.samplename = None
            self.__backgroundImage = None

            print('Note: sample name and one sun calibration results have been reset to None')

        self.__dataBuffer = []

        return fpath

    ### calibration methods

    def findOneSun(self, jsc, area):
        ### finds fraction laser power for which measured jsc = target value from solar simulator JV testing.
        # jsc: short circuit current density in mA/cm^2 (positive)
        # area: active area cm^2
        if jsc < 1:
            print('Please provide jsc in units of mA/cm^2, and area in units of cm^2')
            return False

        isc = jsc * area / 1000 	#negative total current in amps, since kepco will be measuring total photocurrent as amps

        laserpowers = np.linspace(0,0.9, 10)[3:]	#array([0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9])
        print(laserpowers)
        result = self.keithley.set_voltage(voltage = 0, onoff = False)

        laserjsc = np.zeros(len(laserpowers))

#         self.cni.set(laser_curr = laserpowers[0])		#set to first power before turning on laser
#         self.cni.on()

        self.jenoptik.set(power = laserpowers[0])		#set to first power before turning on laser
        self.jenoptik.on()

        for idx, power in enumerate(laserpowers):
            self.jenoptik.set(power = power)
            time.sleep(0.5)
            time.sleep(self.saturationtime)
            _, _isc = self.keithley.jsc(printed = False)  
            laserjsc[idx] = _isc
            print(laserjsc[idx])
        self.jenoptik.off()
        # self.keithley.off()
        #pdb.set_trace()

        pfit = np.polyfit(laserjsc, laserpowers, 2)
        p = np.poly1d(pfit)	#polynomial fit object where x = measured jsc, y = laser power applied

        self._sampleOneSun = p(isc)
        self._sampleOneSunSweep = [laserpowers, laserjsc]
        self._sampleOneSunJsc = jsc

        #pdb.set_trace()

        return p(isc), laserpowers, laserjsc	#return laser power to match target jsc

    
    def takeRseMeas(self, vmpp, voc, vstep = 0.005):
        # generate list of biases spanning from vmpp to at least voc, with intervals of vstep
        biases = [vmpp + (voc-vmpp)/2]
        while biases[-1] < voc + 0.07:		#go to 70 mV (about 10% of starting Voc) higher voltage than Voc, better fitting/calibration constant is linear
            biases.append(biases[-1] + vstep)

        with tqdm(total = len(biases), desc = 'Rse EL', leave = False) as pb:
            for bias in biases[0:-1]:	#measure all but last with lastmeasurement = True (doesnt turn kepco off between measurements). Last measurement is normal
                self.setMeas(bias = bias, laserpower = 0, note = 'part of Rse measurement series')
                self.takeMeas(lastmeasurement = False)
                pb.update(1)

            self.setMeas(bias = biases[-1], laserpower = 0, note = 'part of Rse measurement series')
            self.takeMeas(lastmeasurement = True)		
            pb.update(1)

    def takePLIVMeas(self, vmpp, voc, jsc, area, rev_PLIV = False, only_OneSun = False, num_imgs = 5):
        ### Takes images at varied bias and illumination for PLIV fitting of cell parameters
        ### based on https://doi.org/10.1016/j.solmat.2012.10.010

        if self._sampleOneSun is None:
            self.findOneSun(jsc = jsc, area = area)		# calibrate laser power to one-sun injection by matching jsc from solar simulator measurement

        # full factorial imaging across voltage (vmpp - voc) and illumination (0.2 - 1.0 suns). 25 images
        allbiases = np.append(0,np.linspace(vmpp, voc, num_imgs))		#range of voltages used for image generation (including short-circuit image at each intensity)
        allsuns = np.linspace(0.2, 1, 5) 			#range of suns (pl injection) used for image generation
        
        ### the below line is added by ZJD to accomodate the PSK laser while making minimal modification to the Si PLIV code
        ### the PSK laser (CNI) has a power onset current at:
        ### laser_curr = 0.3 * max laser_curr, the suns is 0,i.e. there's no light, and when 
        ### laser_curr = 1 * max laser_curr, the suns is 1. 
        ### allsuns * 0.7 + 0.3 is to accomodate for this relationship, so when we have 0.2 suns input, we actually input 0.2*0.7+0.3 = 0.44 laser_curr into the laser to find the correct suns value
        ### then the array is multiplied by self._sampleOneSun to accomodate for the 1 sun value of the module itself
        ### self._laserIntensityCorrection is applied to the (allsuns * 0.7 + 0.3) * self._sampleOneSun, so that we take out of any values that can't be read by the laser machine, 
        ### and swap it to a number that is slightly above the unreadable region of the laser. Details see the function itself.
        ### The array is then divided by self._sampleOneSun, because in c.setMeas, the input suns is multiplied by self._sampleOneSun
        ### This is designed by the original Si PLIV. To avoid changing too much of the old code, (but if there's time in the future, we can make the code more rigirous)
        ### we just divide the "procossed" laser input values by self._sampleOneSun, so that in c.setMeas, suns can be used to multiply the self._sampleOneSun
        
        allsuns = self._laserIntensityCorrection((allsuns * 0.7 + 0.3) * self._sampleOneSun) / self._sampleOneSun
        if only_OneSun:
            allsuns = np.array([allsuns[-1]])
        
        if rev_PLIV:
            allbiases = np.flip(allbiases)
            allsuns = np.flip(allsuns)
        
        
        self.setMeas(bias = 0, suns = 1, temperature = 25, note = 'PLIV - open circuit PL image')#???????? no voc mode?
        # this should work, something else is going wrong occasionally.
#         self.kepco.set(current = 0) # does not work as takeMeas will reset the voltage setting based on setMeas parameters. Better to set voc directly in setMeas.
        #pdb.set_trace()
        self.takeMeas(lastmeasurement = False, VocMode = True)##???????? voc mode here?
        time.sleep(self._scanSpeed_timestop)#????????
        with tqdm(total = allbiases.size * allsuns.size, desc = 'PLIV', leave = False) as pb:
            for suns in allsuns:
                for bias in allbiases:
                    self.setMeas(bias = bias, suns = suns, temperature = 25, note = 'PLIV')
                    self.takeMeas(lastmeasurement = False, VocMode = False)
                    time.sleep(self._scanSpeed_timestop)#????????
                    pb.update(1)


        self.cni.off()	#turn off the laser and kepco
        self.__laserON = False
        self.keithley.off()
        self.__keithleyON = False
    
    ### helper functions
    def _waitForTemperature(self):
        refreshDelay = 0.5	#how long to wait between temperautre checks, in seconds
        reachedTemp = False

        startTime = time.time()
        while (not reachedTemp) and (time.time() - startTime <= self.maxSoakTime):
            currentTemp = self.tec.getTemperature()
            if np.abs(currentTemp - self.temperature) <= self.temperatureTolerance:
                reachedTemp = True
            else:
                time.sleep(refreshDelay)

        if not reachedTemp:
            print('Did not reach {0} C within {1} seconds: starting measurement anyways.'.format(self.temperature, self.maxSoakTime))

        return True
    def _getOpticalPower(self):
        '''
        reads signal from photodetector, converts to optical power using calibration vs thorlabs Si power meter (last checked 2019-08-20)
        calibrationFit = [-0.1145, 9.1180]; #polyfit of detector reading vs (Si power meter / detector reading), 2019-08-20
        '''

        voltage, _, _ = self.daq3.acquire()
        power = voltage/1000/0.311/0.13*1000 / 54 
        #Pd readout / Resistor / Reponsivity / pd active area in cm2 * (1000mW/W), ZJD 2023/11/13
        
        #* (calibrationFit[0]*voltage + calibrationFit[1])	#measured optical power, units of mW/cm^2

        return power

    def _backgroundCorrection(self, img):
        img = img - self.__backgroundImage
        img[img<0] = 0

        return img
    
    def _laserIntensityCorrection(self, array):
        '''
        Created this function to avoid laser current range 248～255, 503～511, 758～767
        Because this laser controller has problems (defect from manufacturing). Sad T^T
        '''
        # Multiply the array by x and then by 1000
        CNI_input = array * 1000

        # Update values in the specified ranges
        CNI_input[(CNI_input >= 248) & (CNI_input <= 255)] = 256
        CNI_input[(CNI_input >= 503) & (CNI_input <= 511)] = 512
        CNI_input[(CNI_input >= 758) & (CNI_input <= 767)] = 768
        
        # Return the new array
        array = CNI_input / 1000
        
        return array

    def _intensity_mapping_snaking(self): 
        '''
        Created this function to do intensity mapping using opfo.  
        ZJD 2024/03/28
        '''
        target_x_coor, target_y_coor = self.opfo.target_x_coor, self.opfo.target_y_coor
        pixel_pitch = self.opfo.pixel_pitch
        
        x_mappin_diameter, y_mappin_diameter = self.opfo.x_mappin_diameter, self.opfo.y_mappin_diameter
        map_name = self.opfo.map_name

        x_map_half_radius, y_map_half_radius = x_mappin_diameter/2, y_mappin_diameter/2

        #move to bottom left:
        x_start = target_x_coor - x_map_half_radius
        y_start = target_y_coor + y_map_half_radius

        self.opfo.xmove_to_coor(x_start)
        self.opfo.ymove_to_coor(y_start)

        ##generate an empty grid
        x_grid = int(x_mappin_diameter/pixel_pitch)
        y_grid = int(y_mappin_diameter/pixel_pitch)
        matrix = np.full((x_grid,y_grid, 5), np.nan) 

        flip = 1 #fwd
        x_position_tracker = x_start
        y_position_tracker = y_start

        for y_ in range(y_grid):
            for x_ in range(x_grid):
                if x_ != x_grid-1:
                    avg, std, data = self.daq3.acquire()
                    matrix[x_, y_, 0], matrix[x_, y_, 1], matrix[x_, y_, 2], matrix[x_, y_, 3], matrix[x_, y_, 4] = x_position_tracker, y_position_tracker, x_, y_, avg

                    if flip == 1:

                        pitch_to_move = pixel_pitch

                    if flip == -1:

                        pitch_to_move = -pixel_pitch

                    self.opfo.xmove(pitch_to_move)

                    slptime = pixel_pitch/self._actual_speed * 1.3
                    time.sleep(slptime)

                    x_position_tracker = x_position_tracker + pitch_to_move
                else:
                    avg, std, data = self.daq3.acquire()
                    matrix[x_, y_, 0], matrix[x_, y_, 1], matrix[x_, y_, 2], matrix[x_, y_, 3], matrix[x_, y_, 4] = x_position_tracker, y_position_tracker, x_, y_, avg
            if pixel_pitch < 10:
                time.sleep(0.8)
            self.opfo.ymove(-pixel_pitch)
            slptime = pixel_pitch/self._actual_speed * 1.15

            avg, std, data = self.daq3.acquire()
            time.sleep(slptime)

            y_position_tracker = y_position_tracker - pixel_pitch

            flip = flip * -1 #flip snaking direction
        np.save(f'{os.path.join(datafolder, map_name)}.npy', matrix)
        print(f'saved to {os.path.join(datafolder, map_name)}.npy')

        return matrix

#     def _intensity_mapping_noSnaking(self, datafolder = None):
#         if datafolder is None:
#             datafolder = datafolder

#         target_x_coor = self.opfo.target_x_coor
#         target_y_coor = self.opfo.target_y_coor
#         pixel_pitch = self.opfo.pixel_pitch
#         x_mappin_diameter = self.opfo.x_mappin_diameter
#         y_mappin_diameter = self.opfo.y_mappin_diameter
#         map_name = self.opfo.map_name

#         x_map_half_radius = x_mappin_diameter/2
#         y_map_half_radius = y_mappin_diameter/2

#         #move to bottom left:
#         x_start = target_x_coor - x_map_half_radius
#         # if x_start < 0 or target_x_coor + x_map_half_radius > 200:
#         #     print('x out of bound for the xy stage, aborting intensity mapping')

#         y_start = target_y_coor + y_map_half_radius
#         # if y_start > 200 or target_y_coor - x_map_half_radius < 0:
#         #     print('y out of bound for the xy stage, aborting intensity mapping')
#         #     return

#         self.opfo.xmove_to_coor(x_start)
#         self.opfo.ymove_to_coor(y_start)

#         ##generate an empty grid
#         x_grid = int(x_mappin_diameter/pixel_pitch)
#         y_grid = int(y_mappin_diameter/pixel_pitch)
#         matrix = np.full((x_grid,y_grid), None, dtype=object) 

#         x_counter = 0
#         y_counter = 0

# #             avg, std, data = self.daq3.acquire()
# #             time.sleep(1)
# #             matrix[0][0] = avg

#         for y_ in range(y_grid):
#             for x_ in range(x_grid):
#                 print('x', x_, 'y', y_, '\n')
#                 if x_ == 0 and y_ == 0:
#                     avg, std, data = self.daq3.acquire()
#                     print(x_, y_, avg)
#                     time.sleep(1)
# #                         matrix[y_][x_] = avg
#                     matrix[x_][y_] = avg
# #                         print('initial point recorded')
#                 else:
#                     self.opfo.xmove(pixel_pitch)
#                     slptime = pixel_pitch/self._actual_speed * 1.15
#                     time.sleep(slptime)
#                     avg, std, data = self.daq3.acquire()
#                     print(avg)
#                     time.sleep(1)
# #                         matrix[y_][x_] = avg
#                     matrix[x_][y_] = avg

#                     x_counter += 1
#                     print(x_, y_, avg)
#             self.opfo.xmove_to_coor(x_start)
#             self.opfo.ymove(-pixel_pitch)
#             slptime = pixel_pitch/self._actual_speed * 1.15
#             y_counter += 1

#         matrix_plot = matrix.astype(float)
#         plt.imshow(matrix_plot, vmin = 0.3, vmax = 0.5)
        
    
#         plt.colorbar()
#         plt.savefig(os.path.join(datafolder, f'{map_name}.jpg'))
#         plt.close()
#         np.savetxt(f'{os.path.join(datafolder, map_name)}.txt', matrix_plot)

#         return matrix_plot
        
    
c = control()

mcculw APT did not load properly - if needed, ensure that DLL has been installed!
CNI connected
DAQ3 conected
OpFo connected
TEC controller connected
